In [7]:
import pandas
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [128]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = pandas.read_csv(url, names=names)

In [4]:
dataset.head(2)

,sepal-length,sepal-width,petal-length,petal-width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


In [129]:
dataset.shape

(150, 5)

In [130]:
array = dataset.values
X = array[:,0:4]
Y = array[:,4]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = \
model_selection.train_test_split(X, 
                                 Y, 
                                 test_size=validation_size, 
                                 random_state=seed)

In [103]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

In [13]:
# evaluate each model in turn
seed = 7
scoring = 'accuracy'
results = []
names = []
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=seed)
	cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

LR: 0.966667 (0.040825)
LDA: 0.975000 (0.038188)
KNN: 0.983333 (0.033333)
CART: 0.975000 (0.038188)
NB: 0.975000 (0.053359)
SVM: 0.991667 (0.025000)


In [14]:
# Make predictions on validation dataset
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)
predictions = knn.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9
[[ 7  0  0]
 [ 0 11  1]
 [ 0  2  9]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         7
Iris-versicolor       0.85      0.92      0.88        12
 Iris-virginica       0.90      0.82      0.86        11

    avg / total       0.90      0.90      0.90        30



# Wrong Label Correction

In [98]:
import random
from sklearn.utils import shuffle

In [143]:
def load_dataset():
    url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
    names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
    dataset = pandas.read_csv(url, names=names)
    return dataset

In [173]:
dataset = load_dataset()

In [119]:
dataset.head(2)

,sepal-length,sepal-width,petal-length,petal-width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


In [100]:
labels = list(dataset['class'].unique())
labels

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

In [101]:
dataset = shuffle(dataset)
dataset = dataset.reset_index(drop=True)

In [87]:
labels = list(dataset['class'].unique())
def make_wrong(dataset, num_of_wrongs):
    change_indexes = random.sample(range(0,len(dataset)+1), num_of_wrongs)
    print('change_indexes :\n', change_indexes)
    trues = []
    wrongs = []
    for i in change_indexes:
        true_label = dataset.at[i , 'class']
        trues.append(true_label)
        wrong_label = random.choice([i for i in labels if i != true_label])
        wrongs.append(wrong_label)
        dataset.at[i , 'class'] = wrong_label
    print('trues :\n', trues)
    print('wrongs :\n', wrongs)
    return dataset

In [156]:
def form_models():
    models = {}
    models['LR'] = LogisticRegression(solver='liblinear', multi_class='ovr')
    models['LDA'] = LinearDiscriminantAnalysis()
    models['KNN'] = KNeighborsClassifier()
    models['CART'] = DecisionTreeClassifier()
    models['NB'] = GaussianNB()
    models['SVM'] = SVC(gamma='auto')
    return models
m = form_models()

In [157]:
len(m)

6

In [ ]:
num_of_features = 4
array = dataset.values
X = array[:,0:num_of_features]
Y = array[:,num_of_features]

In [149]:
shuffled_dataset = shuffle(dataset)

In [150]:
shuffled_dataset[:10]

,sepal-length,sepal-width,petal-length,petal-width,class
29,4.7,3.2,1.6,0.2,Iris-setosa
125,7.2,3.2,6.0,1.8,Iris-virginica
55,5.7,2.8,4.5,1.3,Iris-versicolor
66,5.6,3.0,4.5,1.5,Iris-versicolor
131,7.9,3.8,6.4,2.0,Iris-virginica
31,5.4,3.4,1.5,0.4,Iris-setosa
63,6.1,2.9,4.7,1.4,Iris-versicolor
87,6.3,2.3,4.4,1.3,Iris-versicolor
10,5.4,3.7,1.5,0.2,Iris-setosa
37,4.9,3.1,1.5,0.1,Iris-setosa


In [151]:
t = shuffled_dataset[:10]

In [153]:
X_test = t[t.columns[0:4]]
X_test

,sepal-length,sepal-width,petal-length,petal-width
29,4.7,3.2,1.6,0.2
125,7.2,3.2,6.0,1.8
55,5.7,2.8,4.5,1.3
66,5.6,3.0,4.5,1.5
131,7.9,3.8,6.4,2.0
31,5.4,3.4,1.5,0.4
63,6.1,2.9,4.7,1.4
87,6.3,2.3,4.4,1.3
10,5.4,3.7,1.5,0.2
37,4.9,3.1,1.5,0.1


In [155]:
for i, index in enumerate(list(X_test.index)):
    print(i,index)

0 29
1 125
2 55
3 66
4 131
5 31
6 63
7 87
8 10
9 37


In [161]:
from collections import defaultdict, Counter

In [162]:
d = defaultdict(list)
d[1].append('a')
d

defaultdict(list, {1: ['a']})

In [166]:
d[1].append('a')
d[1].append('b')

In [279]:
t = pandas.read_csv('/Users/muratyalcin/Downloads/train.csv')
t.head(2)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
import pandas, random
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from collections import defaultdict, Counter
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

class correct_labels:
    
    def __init__(self,
                 dataset,
                 label_column_name:str, 
                 num_of_wrongs, 
                 repeats, 
                 split_rate = 4, # 3/4 train, 1/4 predict
                 iris = None,
                 mnist = None):
        self.dataset = dataset
        self.label_column_name = label_column_name
        self.split_rate = split_rate 
        self.num_of_wrongs = num_of_wrongs
        self.repeats = repeats
        self.models = self.form_models()
        #if iris:
        #    assert mnist is None
        #    self.dataset = self.load_iris_dataset()
        #if mnist:
        #    self.dataset = self.load_mnist_dataset()
        self.num_of_features = self.dataset.shape[1]-1
        self.labels = list(self.dataset[label_column_name].unique())
        self.num_of_labels = len(self.labels)
        
    
    def load_iris_dataset(self):
        url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
        names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
        dataset = pandas.read_csv(url, names=names)
        return dataset
    
    def load_mnist_dataset(self):
        t = pandas.read_csv('/Users/muratyalcin/Downloads/train.csv')
        cols = list(t.columns)
        cols = cols[1:] + [cols[0]]
        dataset = t[cols]
        return dataset
    
    def shuffle_dataset(self, dataset):
        shuffled_dataset = shuffle(dataset)
        #shuffled_dataset = shuffled_dataset.reset_index(drop=True)
        return shuffled_dataset
        
    def make_wrong(self, dataset):
        change_indexes = random.sample(range(0,len(dataset)+1),
                                       self.num_of_wrongs)
        trues = []
        wrongs = []
        wrong_dataset = dataset.copy()
        for i in change_indexes:
            true_label = self.dataset.at[i , self.label_column_name]
            trues.append(true_label)
            wrong_label = random.choice([i for i in self.labels if i != true_label])
            wrongs.append(wrong_label)
            wrong_dataset.at[i , self.label_column_name] = wrong_label
        return wrong_dataset, trues, wrongs, change_indexes
    
    def dataset_train_test_split(self, dataset):
        split_point = int(len(dataset)/self.split_rate)
        train_data = dataset[split_point:]
        test_data = dataset[:split_point]
        return train_data, test_data, split_point
    
    def df_to_vector(self, df):
        return df.values
        
    def x_y_split_vector(self, vector):
        X = vector[:,0:self.num_of_features]
        y = vector[:,self.num_of_features]
        return X, y 
        
    def form_models(self):
        models = {}
        #models['LR'] = LogisticRegression(solver='liblinear', multi_class='ovr')
        #models['LDA'] = LinearDiscriminantAnalysis()
        #models['KNN'] = KNeighborsClassifier()
#         models['CART'] = DecisionTreeClassifier()
#         models['RF'] = RandomForestClassifier()
        #models['NB'] = GaussianNB()
        #models['SVM'] = SVC(gamma='auto')
        #models['baseline'] = self.baseline_model()
        models['CNN'] = self.create_cnn_model()
        return models
    
    def create_cnn_model(self):
        # Set the CNN model 
        # my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

        model = Sequential()
        model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                         activation ='relu', input_shape = (28,28,1)))
        model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                         activation ='relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                         activation ='relu'))
        model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                         activation ='relu'))
        model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
        model.add(Dropout(0.25))


        model.add(Flatten())
        model.add(Dense(256, activation = "relu"))
        model.add(Dropout(0.5))
        model.add(Dense(10, activation = "softmax"))
        
        # Define the optimizer
        optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
        # Compile the model
        model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
        return model
        
    def baseline_model(self):
        # create model
        model = Sequential()
        model.add(Dense(self.num_of_features, input_dim=self.num_of_features, kernel_initializer='normal', activation='relu'))
        model.add(Dense(self.num_of_labels, kernel_initializer='normal', activation='softmax'))
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
       

    def fit_cnn(self, model, X_train, Y_train, X_val, Y_val):
        epochs = 1 # Turn epochs to 30 to get 0.9967 accuracy
        batch_size = 64

        # without data augmentation
        model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, 
        validation_data = (X_val, Y_val), verbose = 2)

        return model     
    
    def fit_(self, model, X_train, y_train):
        model.fit(X_train, y_train)
        return model
    
    def predict_(self, model,  X_test):
        predictions = model.predict(X_test)
        return predictions
        
    def multi_model_predict(self, X_train, y_train, X_test):
        preds = []
        for model in list(self.models.values()):
            model = self.fit_(model, X_train, y_train)
            predictions = self.predict_(X_test)
            preds.append(predictions)
        return preds
    
    def multi_model_predict_cnn(self, X_train, Y_train, X_val, Y_val, X_test):
        preds = []
        for model in list(self.models.values()):
            model = self.fit_cnn(model, X_train, Y_train, X_val, Y_val)
            predictions = self.predict_(model, X_test)
            print('predictions before : \n', predictions)
            # Convert predictions classes to one hot vectors 
            predictions = np.argmax(predictions,axis = 1) 
            print('predictions after : \n', predictions)
            preds.append(predictions)
        return preds
    
    
    def handle_tracker(self, tracker, wrong_dataset):
        wrong_data_labels = list(wrong_dataset[self.label_column_name])
        assert len(list(self.dataset[self.label_column_name])) == \
        len(list(wrong_dataset[self.label_column_name]))
        item_preds = defaultdict()
        model_guess = []
        for i in range(len(self.dataset)):
            if tracker[i]:
                item_preds[i] = max(Counter(tracker[i]), key=Counter(tracker[i]).get) 
                model_guess.append(max(Counter(tracker[i]), key=Counter(tracker[i]).get))
            else:
                item_preds[i] = wrong_data_labels[i]
                model_guess.append(wrong_data_labels[i])
        return item_preds, model_guess
   
    def compare(self, model_guess):
        actuals = list(self.dataset[self.label_column_name])
        predicted = model_guess 
        corrects = [i for i, j in enumerate(zip(actuals, model_guess)) if j[0] == j[1]]
        wrongs = [i for i, j in enumerate(zip(actuals, model_guess)) if j[0] != j[1]]
        return corrects, wrongs
 
    def evaluate(self, corrects, change_indexes, wrongs):
        return {
            'data length' : len(self.dataset),
            'split rate' : self.split_rate,
            'repeats' : self.repeats,
            'total wrongs start' : self.num_of_wrongs,
            'number of corrects' : len(corrects),
            'number of wrongs' : len(wrongs),
            'number of wrong indexes'  : len(change_indexes),
            'number of corrected' : len(set(change_indexes) & set(corrects)),
            'number of missed' : len(set(change_indexes) & set(wrongs)), 
            'number of wronged' : len((set(change_indexes) | set(wrongs)) - set(change_indexes))   
        }
        
    def correct_wrong_labels(self):
        tracker = defaultdict(list)
        wrong_dataset, trues, wrongs, change_indexes = self.make_wrong(self.dataset)
        for i in range(self.repeats):
            dataset = self.shuffle_dataset(wrong_dataset)
            train_data, test_data, split_point = self.dataset_train_test_split(wrong_dataset)
            train_data_ = self.df_to_vector(train_data)
            test_data_ = self.df_to_vector(test_data)
            X_train, y_train = self.x_y_split_vector(train_data_)
            X_test, y_test = self.x_y_split_vector(test_data_) 
            
            preds = self.multi_model_predict(X_train, y_train, X_test)
            num_models = len(self.models)
            assert len(preds[0]) == split_point
            y_indexes = list(test_data.index)
            for x in range(num_models):
                for i, index in enumerate(y_indexes):
                    tracker[index].append(preds[x][i])
        item_preds, model_guess = self.handle_tracker(tracker, wrong_dataset)
        corrects, wrongs = self.compare(model_guess)
        result = self.evaluate(corrects, change_indexes, wrongs)
        print('result : ', result) 
        return result
    
    
    def correct_wrong_labels_cnn(self):
        tracker = defaultdict(list)
        wrong_dataset, trues, wrongs, change_indexes = self.make_wrong(self.dataset)
        for i in range(self.repeats):
            print(f'processing {i}/{self.repeats}')
            dataset = self.shuffle_dataset(wrong_dataset)
            train_data, test_data, split_point = self.dataset_train_test_split(wrong_dataset)
            # Drop 'label' column
            X_train = train_data.drop(labels = ["label"],axis = 1) 
            Y_train = train_data["label"]

            X_test = test_data.drop(labels = ["label"],axis = 1) 
            Y_test = test_data["label"]
            # Normalize the data
            X_train = X_train / 255.0
            X_test = X_test / 255.0

            # Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
            X_train = X_train.values.reshape(-1,28,28,1)
            X_test = X_test.values.reshape(-1,28,28,1)
            
            # Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
            Y_train = to_categorical(Y_train, num_classes = 10)
            Y_test = to_categorical(Y_test, num_classes = 10)
            
            # Split the train and the validation set for the fitting
            X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1)

            
#             # Set a learning rate annealer
#             learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
#                                             patience=3, 
#                                             verbose=1, 
#                                             factor=0.5, 
#                                             min_lr=0.00001)

#             datagen = ImageDataGenerator(
#                     featurewise_center=False,  # set input mean to 0 over the dataset
#                     samplewise_center=False,  # set each sample mean to 0
#                     featurewise_std_normalization=False,  # divide inputs by std of the dataset
#                     samplewise_std_normalization=False,  # divide each input by its std
#                     zca_whitening=False,  # apply ZCA whitening
#                     rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#                     zoom_range = 0.1, # Randomly zoom image 
#                     width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#                     height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#                     horizontal_flip=False,  # randomly flip images
#                     vertical_flip=False)  # randomly flip images

#             datagen.fit(X_train)
            
#             # Fit the model
#             history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
#                               epochs = epochs, validation_data = (X_val,Y_val),
#                               verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
#                               , callbacks=[learning_rate_reduction])
            print('multi model prediction in progress...')
            preds = self.multi_model_predict_cnn(X_train, Y_train, X_val, Y_val, X_test)
            print('multi model prediction completed successfully...')
            num_models = len(self.models)
            assert len(preds[0]) == split_point
            y_indexes = list(test_data.index)
            for x in range(num_models):
                for i, index in enumerate(y_indexes):
                    tracker[index].append(preds[x][i])
        item_preds, model_guess = self.handle_tracker(tracker, wrong_dataset)
        corrects, wrongs = self.compare(model_guess)
        result = self.evaluate(corrects, change_indexes, wrongs)
        print('result : ', result) 
        return result

In [33]:
def start_correct(num_of_wrongs, repeats, split_rate):
    def load_mnist_dataset():
        t = pandas.read_csv('/Users/muratyalcin/Downloads/train.csv')
        cols = list(t.columns)
        cols = cols[1:] + [cols[0]]
        dataset = t[cols]
        return dataset
    num_of_wrongs = num_of_wrongs
    repeats = repeats
    split_rate = split_rate
    results = []
    print('loading dataset...')
    dataset = load_mnist_dataset()
    print('experiment started...')
    for i in num_of_wrongs:
        for j in repeats:
            for k in split_rate:
                cl = correct_labels(dataset = dataset,
                                    label_column_name = 'label', 
                                    num_of_wrongs = i, 
                                    repeats = j, 
                                    split_rate = k,
                                   mnist = True)
                print('\ncombination : \n', (i, j, k) , '\n')
                result = cl.correct_wrong_labels_cnn()
                results.append(result)
    res = pandas.DataFrame(results)
    print(res)

In [ ]:
num_of_wrongs = [100]
repeats = [1]
split_rate = [100]
start_correct(num_of_wrongs, repeats, split_rate)

loading dataset...
experiment started...

combination : 
 (100, 1, 100) 

processing 0/1
multi model prediction in progress...
Train on 37422 samples, validate on 4158 samples
Epoch 1/1


In [3]:
pandas.DataFrame(results)

,data length,number of corrected,number of corrects,number of missed,number of wrong indexes,number of wronged,number of wrongs,repeats,split rate,total wrongs start
0,42000,3,40805,2,5,1193,1195,10,3,5
1,42000,3,40858,12,15,1130,1142,10,3,15


In [263]:
cl = correct_labels('class', num_of_wrongs = 5, repeats = 10000, split_rate = 5)

In [264]:
cl.correct_wrong_labels()

result :  {'data length': 150, 'number of corrects': 148, 'number of wrongs': 2, 'number of wrong indexes': 5, 'number of corrected': 3, 'number of missed': 2, 'number of wronged': 0}


In [3]:
def load():
    import pandas
    t = pandas.read_csv('/Users/muratyalcin/Downloads/train.csv')
    cols = list(t.columns)
    cols = cols[1:] + [cols[0]]
    dataset = t[cols]
    return dataset

In [5]:
d = load()

In [6]:
d.shape

(42000, 785)

In [11]:
cols = list(t.columns)
cols = cols[1:] + [cols[0]]
df = t[cols]
df.head(2)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#from keras.datasets import mnist
#(X_train, y_train), (X_test, y_test) = mnist.load_data()